[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/observation_prediction_bodies.ipynb)

# Transduction from observation to prediction for throwing a dice

In [1]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction_bodies.sh http://josd.github.io/eye/transduction/observation_prediction_dice.sh
! curl -o observation_prediction_bodies.py http://josd.github.io/eye/transduction/observation_prediction_dice.py
! curl -o __init__.py http://josd.github.io/eye/transduction/__init__.py
! curl -o sample_bodies.observation http://josd.github.io/eye/transduction/sample_dice.observation
! chmod +x observation_dice_bodies.sh
% rm -fr ~/t2t_data/observation_prediction_dice
% rm -fr ~/t2t_train/observation_prediction_dice/transformer-transformer_small/

In [2]:
# See the observation_prediction_bodies problem

! pygmentize -g observation_prediction_dice.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionDice(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for throwing a dice."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      throw = random.randint(1, 100000)
      

In [3]:
# See the observation_prediction_bodies script

! pygmentize -g observation_prediction_dice.sh

#!/bin/bash
PROBLEM=observation_prediction_dice
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=1000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample_dice.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample_dice.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
 

In [4]:
# Run the observation_prediction_bodies script

! ./observation_prediction_dice.sh

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module transduction from path /home/jdroo/github.com/josd/eye
[2018-05-13 20:40:06,589] Importing user module transduction from path /home/jdroo/github.com/josd/eye
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction_dice
[2018-05-13 20:40:06,590] Generating problems:
    observation:
      * observation_prediction_dice
INFO:tensorflow:Generating data for observation_prediction_dice.
[2018-05-13 20:40:06,591] Generating data for observation_prediction_dice.
INFO:tensorflow:Generating vocab file: /home/jdroo/t2t_data/observation_prediction_dice/vocab.observation_prediction_dice.16384.subwords
[2018-05-13 20:40:06,591] Generat

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
[2018-05-13 20:40:53,731] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/utils/trainer_lib.py:151: RunConfig.__init__ (from tensorflow.contrib.learn.python.learn.estimators.run_config) is deprecated and will be removed in a future version.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
[2018-05-13 20:40:53,731] schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
[2018-05-13 20:40:53,731] worker_gpu=1
INFO:tensorflow:sync=False
[2018-05-13 20:40:53,732] sync=False
[2018-05-13 20:40:53,732] Schedule=continuous_train_and_eval. Assuming that training is running on a single machine.
INFO:tensorflow:datashard_devices: ['gpu:0']
[2018-05-13 20:40:53,732] datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
[2018-05-13 20:40:53,732] caching_device

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
INFO:tensorflow:Done calling model_fn.
[2018-05-13 20:41:03,889] Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
[2018-05-13 20:41:03,890] Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
[2018-05-13 20:41:05,597] Graph was finalized.
2018-05-13 20:41:05.597780: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-05-13 20:41:05.713849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-05-13 20:41:05.714522: I tensorflow/core/common_ru

INFO:tensorflow:Training model for 100 steps
[2018-05-13 20:42:28,108] Training model for 100 steps
INFO:tensorflow:Reading data files from /home/jdroo/t2t_data/observation_prediction_dice/observation_prediction_dice-train*
[2018-05-13 20:42:28,118] Reading data files from /home/jdroo/t2t_data/observation_prediction_dice/observation_prediction_dice-train*
INFO:tensorflow:partition: 0 num_data_files: 7
[2018-05-13 20:42:28,119] partition: 0 num_data_files: 7
INFO:tensorflow:Calling model_fn.
[2018-05-13 20:42:28,272] Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
[2018-05-13 20:42:31,105] Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
[2018-05-13 20:42:31,107] Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_17182_256.bottom
[2018-05-13 20:42:31,138] Transforming feature 'inputs' with symbol_modality_17182_256.bottom
INFO:tensorflow:Transforming 'targ

INFO:tensorflow:Finished evaluation at 2018-05-13-18:43:59
[2018-05-13 20:43:59,773] Finished evaluation at 2018-05-13-18:43:59
INFO:tensorflow:Saving dict for global step 200: global_step = 200, loss = 1.1194928, metrics-observation_prediction_dice/accuracy = 0.6666667, metrics-observation_prediction_dice/accuracy_per_sequence = 0.0, metrics-observation_prediction_dice/accuracy_top5 = 0.8907936, metrics-observation_prediction_dice/approx_bleu_score = 0.02204563, metrics-observation_prediction_dice/neg_log_perplexity = -1.3348752, metrics-observation_prediction_dice/rouge_2_fscore = 0.0, metrics-observation_prediction_dice/rouge_L_fscore = 0.66666657
[2018-05-13 20:43:59,773] Saving dict for global step 200: global_step = 200, loss = 1.1194928, metrics-observation_prediction_dice/accuracy = 0.6666667, metrics-observation_prediction_dice/accuracy_per_sequence = 0.0, metrics-observation_prediction_dice/accuracy_top5 = 0.8907936, metrics-observation_prediction_dice/approx_bleu_score = 0.0

...

INFO:tensorflow:Running local_init_op.
[2018-05-13 20:51:50,905] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-05-13 20:51:50,957] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 701 into /home/jdroo/t2t_train/observation_prediction_dice/transformer-transformer_small/model.ckpt.
[2018-05-13 20:51:59,266] Saving checkpoints for 701 into /home/jdroo/t2t_train/observation_prediction_dice/transformer-transformer_small/model.ckpt.
INFO:tensorflow:step = 700, loss = 0.56065744
[2018-05-13 20:52:00,054] step = 700, loss = 0.56065744
INFO:tensorflow:Saving checkpoints for 800 into /home/jdroo/t2t_train/observation_prediction_dice/transformer-transformer_small/model.ckpt.
[2018-05-13 20:52:58,163] Saving checkpoints for 800 into /home/jdroo/t2t_train/observation_prediction_dice/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.5486368.
[2018-05-13 20:52:58,973] Loss for final step: 0.5486368.
INFO:tensorflow:Evaluating

INFO:tensorflow:Graph was finalized.
[2018-05-13 20:53:22,092] Graph was finalized.
2018-05-13 20:53:22.092921: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-05-13 20:53:22.092957: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 20:53:22.092964: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-05-13 20:53:22.092970: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-05-13 20:53:22.093136: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 1502 MB memory) -> physical GPU (device: 0, name: GeForce 840M, pci bus id: 0000:03:00.0, compute capability: 5.0)
INFO:tensorflow:Restoring parameters from /home/jdroo/t2t_train/observation_prediction_dice/transformer-transformer_small/model.ckpt-800
[2018-05-13 20:53:22,094] Restoring parameters from /ho

INFO:tensorflow:Using variable initializer: uniform_unit_scaling
[2018-05-13 20:54:46,462] Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_17182_256.bottom
[2018-05-13 20:54:46,493] Transforming feature 'inputs' with symbol_modality_17182_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_17182_256.targets_bottom
[2018-05-13 20:54:46,614] Transforming 'targets' with symbol_modality_17182_256.targets_bottom
INFO:tensorflow:Building model body
[2018-05-13 20:54:46,624] Building model body
INFO:tensorflow:Transforming body output with symbol_modality_17182_256.top
[2018-05-13 20:54:49,225] Transforming body output with symbol_modality_17182_256.top
INFO:tensorflow:Base learning rate: 0.200000
[2018-05-13 20:54:49,333] Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 8088064
[2018-05-13 20:54:49,341] Trainable Variables Total size: 8088064
INFO:tensorflow:Using optimizer Adam

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module transduction from path /home/jdroo/github.com/josd/eye
[2018-05-13 20:56:22,959] Importing user module transduction from path /home/jdroo/github.com/josd/eye
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
[2018-05-13 20:56:23,098] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/utils/trainer_lib.py:151: RunConfig.__init__ (from tensorflow.contrib.learn.python.learn.estimators.run_config) is deprecated and will be removed in a future version.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_tra

2018-05-13 20:56:29.790733: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 20:56:29.790761: I tensorflow/core/common_runtime/gpu/gpu_device.cc:929]      0 
2018-05-13 20:56:29.790782: I tensorflow/core/common_runtime/gpu/gpu_device.cc:942] 0:   N 
2018-05-13 20:56:29.790953: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 1502 MB memory) -> physical GPU (device: 0, name: GeForce 840M, pci bus id: 0000:03:00.0, compute capability: 5.0)
INFO:tensorflow:Restoring parameters from /home/jdroo/t2t_train/observation_prediction_dice/transformer-transformer_small/model.ckpt-1000
[2018-05-13 20:56:29,808] Restoring parameters from /home/jdroo/t2t_train/observation_prediction_dice/transformer-transformer_small/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
[2018-05-13 20:56:30,037] Running local_init_op.
INFO:tensorflow:D

In [5]:
# See the transductions
# For each observation the top 3 predictions are shown with their respective log probability

! pygmentize -g sample_dice.observation
print("->-")
! pygmentize -g sample_dice.prediction

A_THROW t100001
A_THROW t100002
A_THROW t100003
A_THROW t100004
A_THROW t100005
A_THROW t100006
->-
outcome 2	-1.60	outcome 6	-1.61	outcome 1	-1.62
outcome 2	-1.58	outcome 6	-1.61	outcome 1	-1.63
outcome 2	-1.59	outcome 6	-1.61	outcome 1	-1.65
outcome 2	-1.59	outcome 6	-1.61	outcome 1	-1.64
outcome 2	-1.59	outcome 6	-1.60	outcome 1	-1.63
outcome 6	-1.61	outcome 2	-1.63	outcome 1	-1.64
